In [ ]:
import warnings, os
import datetime as dt
import itertools as it
from numpy import nan
import numpy as np
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns

warnings.filterwarnings('ignore')
pd.set_option('display.width',       None)
pd.set_option('display.max_columns', None)

In [ ]:
import MetaTrader5 as mt5
# from vectorbt.portfolio import Portfolio as vbt_Portfolio

# Connection

In [ ]:
RICO_REAL   = { 'login':16331371,   'password':'#000000Zz', 'server':'Rico-PRD'                 }
CLEAR_REAL  = { 'login':1018156111, 'password':'#000000Zz', 'server':'ClearInvestimentos-CLEAR' }

PATH_7 = r'C:\Program Files\MetaTrader 5 _7\terminal64.exe'

assert mt5.initialize(PATH_7, **CLEAR_REAL)

# Variables

In [ ]:
SYMBOLS = ['WIN$'] # ['WDO$']
TFRAMES = ['M5','M15','H1','H4','D1']

# Data

In [ ]:
def MT5_LOAD_DATA(SYMBOLS, TFRAMES, start_pos=0, count=99_999):
    Pipe = []
    for (symbol, TF) in it.product(SYMBOLS, TFRAMES):
        timeframe = { 'M1':mt5.TIMEFRAME_M1, 'M5':mt5.TIMEFRAME_M5, 'M10':mt5.TIMEFRAME_M10, 'M15':mt5.TIMEFRAME_M15, 'M20':mt5.TIMEFRAME_M20, 'M30':mt5.TIMEFRAME_M30, 'H1':mt5.TIMEFRAME_H1, 'H2':mt5.TIMEFRAME_H2, 'H3':mt5.TIMEFRAME_H3, 'H4':mt5.TIMEFRAME_H4, 'H6':mt5.TIMEFRAME_H6, 'H8':mt5.TIMEFRAME_H8, 'H12':mt5.TIMEFRAME_H12, 'D1':mt5.TIMEFRAME_D1, 'W1':mt5.TIMEFRAME_W1 }[TF]
        Raw       = pd.DataFrame(mt5.copy_rates_from_pos(symbol, timeframe, start_pos, count))
        Raw.insert(0, 'a',          Raw.index + 1)
        Raw.insert(1, 'z',          Raw.index - Raw.index.max())
        Raw.insert(2, 'symbol',     symbol)
        Raw.insert(3, 'tf',         TF)
        Raw.insert(4, 'datetime',   pd.to_datetime(Raw['time'], unit='s'))
        Pipe.append(Raw)
    pass
    return pd.concat(Pipe, ignore_index=1)

Raw = MT5_LOAD_DATA(SYMBOLS, TFRAMES)
Raw.head()

In [ ]:
# Raw.to_csv('Storage/Raw.csv', index=False)
Raw = pd.read_csv('Storage/Raw.csv', parse_dates=['datetime'])
Raw.head()

In [ ]:
def FORMAT_SOURCE(Raw):
    Src = pd.DataFrame()
    Src['A']        = Raw['a']
    Src['Z']        = Raw['z']
    Src['Symbol']   = Raw['symbol']
    Src['TF']       = Raw['tf']
    
    Src['Datetime'] = Raw['datetime']
    Src['Date']     = Raw['datetime'].dt.date
    Src['Time']     = Raw['datetime'].dt.time
    
    Src['Ticks']    = Raw['tick_volume']
    Src['Volume']   = Raw['real_volume']
    Src['Price']    = Raw['close']
    
    Src['Open']     = Raw['open']
    Src['High']     = Raw['high']
    Src['Low']      = Raw['low']
    Src['Close']    = Raw['close']

    Src['Change']   = Src['Close'] - Src['Open']
    Src['HL']       = Src['High']  - Src['Low']
    return Src 

Src = FORMAT_SOURCE(Raw)
Src.head()

In [ ]:
Src.groupby(['Symbol','TF'],sort=0)[['Datetime']].first()

In [ ]:
def CALCULATIONS(Src):
    Calc = pd.DataFrame(Src)
    Calc['Change Abs'] = Src['Change'].abs()
    Calc['Change Pos'] = Src['Change'].apply(lambda x: x if (x >= 0) else nan)
    Calc['Change Neg'] = Src['Change'].apply(lambda x: x if (x <  0) else nan)
    return Calc

Calc = CALCULATIONS(Src)
Calc.head()

# Stats

In [ ]:
Calc.groupby(['Symbol','TF'], sort=0)[['Change']].describe().round(0).astype(int)

In [ ]:
Calc.groupby(['Symbol','TF'], sort=0)[['Change Abs']].describe().round(0).astype(int)

In [ ]:
Calc.groupby(['Symbol','TF'], sort=0)[['Change Pos']].describe().round(0).astype(int)

In [ ]:
Calc.groupby(['Symbol','TF'], sort=0)[['Change Neg']].describe().round(0).astype(int)

In [ ]:
Calc.groupby(['Symbol','TF'], sort=0)[['HL']].describe().round(0).astype(int)

In [ ]:
raise Exception('STOP')

# Chart

In [ ]:
for SYMBOL, Asset in Src.groupby('Symbol', sort=0):

    plt.title(f'{SYMBOL} - Candle Size Distribuition', fontweight='bold')

    for TF, Data in Asset.groupby('TF', sort=0): 
        # sns.kdeplot(Data['Change'], label=TF)
        # sns.histplot(Data['Change'], label=TF, stat='count', bins=100, element='step', fill=False)
        plt.hist(Data['Change'], bins=100, label=TF, histtype='step')


    plt.ylabel('Frequency')
    plt.xlabel('Candle Size')

    plt.yscale('log', base=10)
    plt.gca().yaxis.set_major_formatter(mpl.ticker.ScalarFormatter())

    plt.legend()
    plt.tight_layout()
    plt.show()